In [ ]:
# @markdown connect google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title Dependencies

# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --upgrade
!pip install langchain --upgrade
!pip install langchainhub --upgrade
!pip install chromadb --upgrade
# !pip install qdrant-client --upgrade
!pip install openai --upgrade
# !pip install python-dotenv --upgrade
!pip install tiktoken  --upgrade
!pip install Flask --upgrade
!pip install flask-ngrok2 --upgrade


In [ ]:
#@title Install bore

%cd /content

!wget https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz -O bore.tar.gz
!tar -xvf bore.tar.gz
!rm bore.tar.gz
!mv ./bore /usr/local/bin/bore
!chmod +x /usr/local/bin/bore

script = """
until bore local $1 --to bore.pub; do
  echo "bore tunnel crashed with exit code $?. Respawning.." >&2
  sleep 1
done
"""


with open('/content/bore.sh', 'w') as bore:
    bore.write(script)

!chmod +x ./bore.sh

%cd /content


In [ ]:
# @title Imports

import os
import getpass

from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.prompts import PromptTemplate

from langchain.chains import LLMChain, ConversationalRetrievalChain, RetrievalQA
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferMemory

from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain.vectorstores import Chroma, Qdrant


In [ ]:
# @title Set OpenAI API Key

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")


In [ ]:
# @title Set Model

model = "gpt-3.5-turbo" # @param ["gpt-3.5-turbo", "gpt-4"]


In [ ]:
# @title Load Data

loader = DirectoryLoader('/content/drive/MyDrive/CyberCup23H2/embed_src', glob='*.txt', loader_cls=TextLoader)
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=64, chunk_overlap=8)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(texts, embeddings)

# vectorstore = Qdrant.from_documents(texts, embeddings, path="/content/drive/MyDrive/CyberCup23H2/qdrant_db", collection_name="water")

In [ ]:
# @title Init LLM

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Make sure the model path is correct for your system!
llm = OpenAI()

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# qa = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), memory=memory)

qa = ConversationalRetrievalChain.from_llm(
  llm=llm,
  retriever=vectorstore.as_retriever(search_kwargs={'k': 2}),
  memory=memory,
  # return_source_documents=True,
  max_tokens_limit=1024
)

# qa = ConversationalRetrievalChain(
#   retriever=vectorstore.as_retriever(search_kwargs={'k': 2}),
#   question_generator=question_generator,
#   combine_docs_chain=doc_chain,
#   memory=memory,
#   max_tokens_limit=512,
# )


In [ ]:
# @title Ask

query = "how many lakes in india" # @param {type:"string"}
result = qa({"question": query})

print(result)
print(result['question'])
print(result['answer'])


In [ ]:
# @title Setup Flask Server
from flask import Flask, request

from flask_ngrok2 import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app=app, auth_token='')

@app.route('/', methods=['POST', 'GET'])
def hello_world():
  print(request)
  print(request.form)
  print(request.method)

  result = qa({"question": request.form["query"]})

  return result["answer"]



In [ ]:
# @title Run API Server
os.environ["FLASK_DEBUG"] = "False"
app.run()

In [ ]:
# @markdown gg
break
!pip install Flask --upgrade
!pip install flask-ngrok2 --upgrade

from flask import Flask, request

from flask_ngrok2 import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app=app, auth_token='1gNClmywUFSKSEG0WscPkJiG6eE_282GBBTd5PuNEqauXo9hj')

@app.route('/', methods=['POST', 'GET'])
def hello_world():
  print(request)
  print(request.form)
  print(request.method)

  return "hallo"

os.environ["FLASK_DEBUG"] = "False"
app.run()